In [2]:
import neptune

neptune.init(
         project_qualified_name='arsde/J21-BDT'
)

neptune.create_experiment(name='skopt-xgb')

https://app.neptune.ai/arsde/J21-BDT/e/JBDT-5


Experiment(JBDT-5)

In [3]:
# !pip install sklearn==0.23.2

In [4]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
import pandas as pd
import numpy as np

In [5]:
path='/mnt/cephfs/ml_data/mc_2021/'

data_real = pd.read_csv('~/J21/processed_data/ProcessedTrainReal/ProcessedTrain_1M.csv.gz'.format(path))
data_real = data_real[data_real['edepR'] < 17.2]

In [6]:
size = int(1e6)
n_feats = len(data_real.columns)-5

X_val = data_real.iloc[:, :-5][size:]
y_val = data_real.iloc[:, -5][size:]

X = data_real.iloc[:, :-5][:size]
y = data_real.iloc[:, -5][:size]

In [9]:
import neptunecontrib.monitoring.skopt as skopt_utils
from xgboost import XGBRegressor
neptune_callback = skopt_utils.NeptuneCallback()

def print_status(optimal_result):
    """Shows the best parameters found and accuracy attained of the search so far."""
    models_tested = pd.DataFrame(tuner.cv_results_)
    best_parameters_so_far = pd.Series(tuner.best_params_)
    print(
        "Model #{}\nBest accuracy so far: {}\nBest parameters so far: {}\n".format(
            len(models_tested),
            np.round(tuner.best_score_, 3),
            tuner.best_params_,
        )
    )
    
    clf_type = tuner.estimator.__class__.__name__
    models_tested.to_csv("xgboost_optimize/" + clf_type + "_cv_results_summary.csv")

model = XGBRegressor(
    n_estimators=5000,
    random_state=22,
    tree_method='gpu_hist'
)

params_space = {
    "learning_rate": Real(0.001, 1.0, "log-uniform"),
    "min_child_weight": Integer(0, 10),
    "max_depth": Integer(6, 14),
    "max_delta_step": Integer(0, 10),
    "subsample": Real(0.5, 1.0, "uniform"),
    "gamma": Real(0, 20, "uniform"),
    "min_child_weight": Integer(0, 5),
    "colsample_bytree": Real(0.7, 1.0, "uniform"),
    "colsample_bylevel": Real(0.7, 1.0, "uniform"),
    "reg_lambda": Real(1e-9, 1000, "log-uniform"),
    "reg_alpha": Real(1e-9, 1.0, "log-uniform"),
    "objective": Categorical(['reg:squarederror', 'reg:squaredlogerror'])
}
    
tuner = BayesSearchCV(
    model,
    params_space,
    n_iter=100,
    cv=5,
    random_state=22,
    n_jobs=4,
    verbose=10,
    scoring="neg_root_mean_squared_error",
    fit_params={
        'eval_set': [(X_val, y_val)],
        'early_stopping_rounds':3
    }
)

results = tuner.fit(
    X, y,
    callback=[
        neptune_callback,
        print_status
    ]
)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.9min remaining:  2.8min
[Parallel(n_jobs=4)]: Done   3 out of   5 | elapsed:  1.9min remaining:  1.3min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  2.3min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  2.3min finished


Model #1
Best accuracy so far: -0.091
Best parameters so far: OrderedDict([('colsample_bylevel', 0.8092793896326872), ('colsample_bytree', 0.8831112179946441), ('gamma', 18.678653173129554), ('learning_rate', 0.03450570695385555), ('max_delta_step', 1), ('max_depth', 9), ('min_child_weight', 4), ('objective', 'reg:squarederror'), ('reg_alpha', 1.081049236893711e-05), ('reg_lambda', 1.043686239159047), ('subsample', 0.8037032523308654)])

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:   32.5s remaining:   48.8s
[Parallel(n_jobs=4)]: Done   3 out of   5 | elapsed:   32.6s remaining:   21.7s
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:   39.7s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:   39.7s finished


Model #2
Best accuracy so far: -0.091
Best parameters so far: OrderedDict([('colsample_bylevel', 0.8092793896326872), ('colsample_bytree', 0.8831112179946441), ('gamma', 18.678653173129554), ('learning_rate', 0.03450570695385555), ('max_delta_step', 1), ('max_depth', 9), ('min_child_weight', 4), ('objective', 'reg:squarederror'), ('reg_alpha', 1.081049236893711e-05), ('reg_lambda', 1.043686239159047), ('subsample', 0.8037032523308654)])

Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

[505]	validation_0-rmse:0.08418
[506]	validation_0-rmse:0.08418
[507]	validation_0-rmse:0.08417
[508]	validation_0-rmse:0.08417
[509]	validation_0-rmse:0.08417
[510]	validation_0-rmse:0.08416
[511]	validation_0-rmse:0.08416
[512]	validation_0-rmse:0.08415
[513]	validation_0-rmse:0.08415
[514]	validation_0-rmse:0.08415
[515]	validation_0-rmse:0.08414
[516]	validation_0-rmse:0.08414
[517]	validation_0-rmse:0.08414
[518]	validation_0-rmse:0.08413
[519]	validation_0-rmse:0.08413
[520]	validation_0-rmse:0.08413
[521]	validation_0-rmse:0.08412
[522]	validation_0-rmse:0.08412
[523]	validation_0-rmse:0.08412
[524]	validation_0-rmse:0.08411
[525]	validation_0-rmse:0.08411
[526]	validation_0-rmse:0.08411
[527]	validation_0-rmse:0.08411
[528]	validation_0-rmse:0.08410
[529]	validation_0-rmse:0.08410
[530]	validation_0-rmse:0.08410
[531]	validation_0-rmse:0.08409
[532]	validation_0-rmse:0.08409
[533]	validation_0-rmse:0.08408
[534]	validation_0-rmse:0.08408
[535]	validation_0-rmse:0.08408
[536]	va

[762]	validation_0-rmse:0.08344
[763]	validation_0-rmse:0.08344
[764]	validation_0-rmse:0.08343
[765]	validation_0-rmse:0.08343
[766]	validation_0-rmse:0.08343
[767]	validation_0-rmse:0.08342
[768]	validation_0-rmse:0.08342
[769]	validation_0-rmse:0.08342
[770]	validation_0-rmse:0.08342
[771]	validation_0-rmse:0.08342
[772]	validation_0-rmse:0.08342
[773]	validation_0-rmse:0.08341
[774]	validation_0-rmse:0.08341
[775]	validation_0-rmse:0.08341
[776]	validation_0-rmse:0.08341
[777]	validation_0-rmse:0.08340
[778]	validation_0-rmse:0.08340
[779]	validation_0-rmse:0.08340
[780]	validation_0-rmse:0.08340
[781]	validation_0-rmse:0.08340
[782]	validation_0-rmse:0.08339
[783]	validation_0-rmse:0.08339
[784]	validation_0-rmse:0.08339
[785]	validation_0-rmse:0.08339
[786]	validation_0-rmse:0.08339
[787]	validation_0-rmse:0.08338
[788]	validation_0-rmse:0.08338
[789]	validation_0-rmse:0.08338
[790]	validation_0-rmse:0.08338
[791]	validation_0-rmse:0.08338
[792]	validation_0-rmse:0.08337
[793]	va

[1018]	validation_0-rmse:0.08297
[1019]	validation_0-rmse:0.08297
[1020]	validation_0-rmse:0.08297
[1021]	validation_0-rmse:0.08297
[1022]	validation_0-rmse:0.08297
[1023]	validation_0-rmse:0.08297
[1024]	validation_0-rmse:0.08297
[1025]	validation_0-rmse:0.08296
[1026]	validation_0-rmse:0.08296
[1027]	validation_0-rmse:0.08296
[1028]	validation_0-rmse:0.08296
[1029]	validation_0-rmse:0.08296
[1030]	validation_0-rmse:0.08296
[1031]	validation_0-rmse:0.08295
[1032]	validation_0-rmse:0.08295
[1033]	validation_0-rmse:0.08295
[1034]	validation_0-rmse:0.08295
[1035]	validation_0-rmse:0.08295
[1036]	validation_0-rmse:0.08295
[1037]	validation_0-rmse:0.08295
[1038]	validation_0-rmse:0.08294
[1039]	validation_0-rmse:0.08294
[1040]	validation_0-rmse:0.08294
[1041]	validation_0-rmse:0.08294
[1042]	validation_0-rmse:0.08294
[1043]	validation_0-rmse:0.08294
[1044]	validation_0-rmse:0.08293
[1045]	validation_0-rmse:0.08293
[1046]	validation_0-rmse:0.08293
[1047]	validation_0-rmse:0.08293
[1048]	val

[1267]	validation_0-rmse:0.08265
[1268]	validation_0-rmse:0.08265
[1269]	validation_0-rmse:0.08265
[1270]	validation_0-rmse:0.08265
[1271]	validation_0-rmse:0.08264
[1272]	validation_0-rmse:0.08264
[1273]	validation_0-rmse:0.08264
[1274]	validation_0-rmse:0.08264
[1275]	validation_0-rmse:0.08264
[1276]	validation_0-rmse:0.08264
[1277]	validation_0-rmse:0.08264
[1278]	validation_0-rmse:0.08264
[1279]	validation_0-rmse:0.08264
[1280]	validation_0-rmse:0.08264
[1281]	validation_0-rmse:0.08264
[1282]	validation_0-rmse:0.08263
[1283]	validation_0-rmse:0.08263
[1284]	validation_0-rmse:0.08263
[1285]	validation_0-rmse:0.08263
[1286]	validation_0-rmse:0.08263
[1287]	validation_0-rmse:0.08263
[1288]	validation_0-rmse:0.08263
[1289]	validation_0-rmse:0.08263
[1290]	validation_0-rmse:0.08262
[1291]	validation_0-rmse:0.08262
[1292]	validation_0-rmse:0.08262
[1293]	validation_0-rmse:0.08262
[1294]	validation_0-rmse:0.08262
[1295]	validation_0-rmse:0.08262
[1296]	validation_0-rmse:0.08262
[1297]	val

[1516]	validation_0-rmse:0.08242
[1517]	validation_0-rmse:0.08241
[1518]	validation_0-rmse:0.08241
[1519]	validation_0-rmse:0.08241
[1520]	validation_0-rmse:0.08241
[1521]	validation_0-rmse:0.08241
[1522]	validation_0-rmse:0.08241
[1523]	validation_0-rmse:0.08241
[1524]	validation_0-rmse:0.08241
[1525]	validation_0-rmse:0.08241
[1526]	validation_0-rmse:0.08241
[1527]	validation_0-rmse:0.08241
[1528]	validation_0-rmse:0.08241
[1529]	validation_0-rmse:0.08241
[1530]	validation_0-rmse:0.08240
[1531]	validation_0-rmse:0.08240
[1532]	validation_0-rmse:0.08240
[1533]	validation_0-rmse:0.08240
[1534]	validation_0-rmse:0.08240
[1535]	validation_0-rmse:0.08240
[1536]	validation_0-rmse:0.08240
[1537]	validation_0-rmse:0.08240
[1538]	validation_0-rmse:0.08240
[1539]	validation_0-rmse:0.08240
[1540]	validation_0-rmse:0.08240
[1541]	validation_0-rmse:0.08239
[1542]	validation_0-rmse:0.08239
[1543]	validation_0-rmse:0.08239
[1544]	validation_0-rmse:0.08239
[1545]	validation_0-rmse:0.08239
[1546]	val

In [11]:
skopt_utils.log_results(results._optim_results[0])

AttributeError: 'Figure' object has no attribute '_make_key'

In [ ]:
# !pip install sklearn==0.24